In [59]:
%load_ext sql
%sql mysql://root:@localhost/rental_nerd
%config SqlMagic.feedback=False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [60]:
query = %sql (\
    SELECT \
    id as 'property_id' \
    FROM \
    properties)

i = query.DataFrame().values


In [64]:
bad_ones = []
f = open('bad_instances.txt', 'w', 0)

ValueError: can't have unbuffered text I/O

In [62]:
for property_id in i:
    property_id = property_id[0]
    if property_id % 1000 == 0:
        print("%i property_id processed" % property_id)
        
    query = %sql (\
        SELECT \
        ptl_1.property_id, \
        count(1) as 'bad_instances'\
        FROM \
        property_transaction_logs AS ptl_1, \
        property_transaction_logs AS ptl_2 \
        WHERE \
        ptl_1.`property_id`= ptl_2.`property_id` AND \
        ptl_1.`transaction_type` = ptl_2.`transaction_type` AND \
        ptl_1.id <> ptl_2.id \
        AND \
        ( \
        ( ptl_1.date_listed IS NOT NULL AND \
        ptl_2.date_listed IS NOT NULL AND \
        ptl_1.date_closed IS NOT NULL AND \
        ptl_1.date_listed < ptl_2.date_listed AND ptl_2.date_listed < ptl_1.date_closed \
        ) \
        OR \
        ( ptl_1.date_listed IS NOT NULL AND \
        ptl_2.date_closed IS NOT NULL AND \
        ptl_1.date_closed IS NOT NULL AND \
        ptl_1.date_listed < ptl_2.date_closed AND \
        ptl_2.date_closed < ptl_1.date_closed \
        ) \
         OR ( \
        ptl_1.date_listed IS NOT NULL AND \
        ptl_1.date_closed IS NOT NULL AND \
        ptl_2.date_listed IS NOT NULL AND \
        ptl_2.date_closed IS NOT NULL AND \
        ptl_1.date_listed > ptl_2.date_listed AND \
        ptl_1.date_closed < ptl_2.date_closed ) ) \
        AND \
        ptl_1.property_id = :property_id )
    
    if(query.DataFrame().bad_instances.ix[0] == 1):
        bad_ones.append(property_id)
        print("Bad property id %i detected" % property_id, file=f,flush=False)
        

In [51]:
print(bad_ones)

[1, 2]


In [63]:
f.close()